# [Fannie Mae Single-Family Loan Performance Data](https://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html)

## About the Dataset

| Data Subset | Size of Compressed Dataset (GB) | Size of Uncompressed Dataset (GB) | Size of Acquistion Dataset (GB) | Size of Performance Dataset (GB) | Number of Loans (Millions) | Number of Performance Records (Billions) |
| ------- | ----------------------------- | ---------------------------- | -------- | ----------| ---------------- | ---------------------------------------- |
| mortgage_2000      | 0.45 | 3.9 | 0.137 | 3.7 | 1.21 | 0.036 |
| mortgage_2000-2001 | 1.9 | 16 | 0.473 | 16 | 4.22 | 0.148 |
| mortgage_2000-2003 | 9.3 | 78 | 1.5 | 76 | 13.5 | 0.743 |
| mortgage_2000-2007 | 15 | 117 | 2.1 | 115 | 19.1 | 1.12 |
| mortgage_2000-2015 | 23 | 192 | 3.8 | 188 | 34.7 | 1.85 |
| mortgage_2000-2016 | 24 | 196 | 4.1 | 192 | 37.0 | 1.89 |

## Import Python Modules

In [23]:
# Import all of the python modules from The Python Standard Library
# required to run the notebook.

import os
import glob
import collections #hello

In [24]:
# Import all standard data science and machine learning python modules 
# required to run the notebook.

import dask
import dask.dataframe
import dask.distributed
#import matplotlib
import numpy
import pandas
import sklearn

In [25]:
# Import NVIDIA's new RAPIDS python modules designed to perform 
# GPU-acclerated data science and machine learning tasks.

#import cudf
#import cuml  #delete

## Configure Notebook

In [26]:
# Define system environment variables
#USER = os.environ['USER']
USER = "caolinnh"

In [27]:
# Define a path to the top-level directory of a local or network
# filesystem that is available and accessible to you on the system 
# where you are running the notebook.

DATA_ROOT_DIR = os.path.join('/oasis/scratch/comet/caolinnh'+ '/temp_project')
print(DATA_ROOT_DIR)



/oasis/scratch/comet/caolinnh/temp_project


In [28]:
# Choose which version of the Fannie Mae Single-Family Loan 
# Performance (SFLP) dataset you wish to analyze.

SFLP_DATASET = 'dataJup'

In [29]:
# Define a path to the directory of the filesystem where you want to 
# download the compressed SFLP dataset, extract it into its uncompressed
# form, and store for your analysis.

SFLP_DATASET_DIR = os.path.join(DATA_ROOT_DIR, 
                                '/oasis/scratch/comet/caolinnh/temp_project',
                                SFLP_DATASET)
print(SFLP_DATASET_DIR)

/oasis/scratch/comet/caolinnh/temp_project/dataJup


In [30]:
# Define the two paths to the subdirectories where the loan acquisition
# and performance datasets will be and/or are stored.

SFLP_SQUEUE_DATA_DIR = os.path.join(SFLP_DATASET_DIR + '/SQUEUE')
SFLP_SACCT_DATA_DIR = os.path.join(SFLP_DATASET_DIR +'/SACCT')
print(SFLP_DATASET_DIR)
print(SFLP_SQUEUE_DATA_DIR)

/oasis/scratch/comet/caolinnh/temp_project/dataJup
/oasis/scratch/comet/caolinnh/temp_project/dataJup/SQUEUE


## Download and Extract Dataset

In [31]:
# Download the compressed SFLP dataset and extract it into its 
# uncompressed form, if it's not already available on the system where
# you're running the notebook.

#SFLP_DATA_ROOT_URL = 'http://rapidsai-data.s3-website.us-east-2.amazonaws.com/notebook-mortgage-data'
if not os.path.exists(SFLP_DATASET_DIR):
    os.makedirs(SFLP_DATASET_DIR)
    os.chdir(SFLP_DATASET_DIR)
    #os.system('wget ' + SFLP_DATA_ROOT_URL + '/' + SFLP_DATASET + '.tgz')
    os.system('tar -xf ' + SFLP_DATASET + '.tgz')
    print('SFLP dataset downloaded and extracted.')
else:
    print('SFLP dataset directory already exists. Check if dataset is already available.')

SFLP dataset directory already exists. Check if dataset is already available.


In [32]:
# Check the contents of SFLP dataset directory.

print(os.listdir(SFLP_DATASET_DIR))

['SQUEUE', 'SACCT']


## Dataset Preparation and Loading

### Set Analysis Interval

In [33]:
# Select the first year and quarter to begin your analysis as well as
# the last year and quarter to end your analysis.

squeue_begin_day = 20190201
squeue_end_day = 20190201
squeue_interval = 1

sacct_start_job = 18456165
sacct_end_job = 20234782
sacct_interval = 1000


In [34]:
# Create two lists of all the quarterly loan acquisition and performance
# data files from the SFLP dataset that are contained within the time 
# interval defined above.

sflp_squeue_files = list()
sflp_sacct_files = list()
dateList = list()
dayList = list()
jobList = list()

for i in range(squeue_begin_day, squeue_end_day + 1, squeue_interval):
    dateList.append(str(i))
                    
print(dateList)
    
for (dirpath, dirnames, filenames) in os.walk(SFLP_SQUEUE_DATA_DIR) :
    for filename in filenames:
        for this in dateList:
        #number = int(filename[11:19])
            if this in str(filename) and (len(filename) > 14):
                dayList.append(str(filename))

print(dayList)


for (dirpath, dirnames, filenames) in os.walk(SFLP_SACCT_DATA_DIR):
    for filename in filenames:
        number = int(filename[:8])
        if (number in range(sacct_start_job, sacct_end_job)):
            jobList.append(number)
print(jobList)
        
    
    # Determine the number of quarters in each year to analyze
   #if (jobID == sflp_begin_year):
      #  sflp_quarters = list(range(sflp_begin_quarter, 5))
  #  elif (sflp_year == sflp_end_year):
  #      sflp_quarters = list(range(1, sflp_end_quarter+1))
  #  else:
 #       sflp_quarters = list(range(1, 5))
        
    # Get absolute path for each quarterly loan acquisition and 
    # performance data file
for day in dayList:
    sflp_squeue_files.extend(
        glob.glob(
            os.path.abspath(
                os.path.join(SFLP_SQUEUE_DATA_DIR, day))))
for job in jobList:
    sflp_sacct_files.extend(
        glob.glob(
            os.path.abspath(
                os.path.join(SFLP_SACCT_DATA_DIR, str(job) + "-I-" + str(sacct_interval) + '.txt'))))
    
    
print(SFLP_SQUEUE_DATA_DIR)
print(SFLP_SACCT_DATA_DIR)

['20190201']
['1549079401-20190201-I-1']
[18456165]
/oasis/scratch/comet/caolinnh/temp_project/dataJup/SQUEUE
/oasis/scratch/comet/caolinnh/temp_project/dataJup/SACCT


### SQUEUE Data

In [35]:
# List all of the files from the SQUEUE dataset to be analyzed
# as well as the size of each file in bytes.

for sflp_squeue_file in sflp_squeue_files:
    print(os.path.basename(sflp_squeue_file), os.stat(sflp_squeue_file).st_size)


1549079401-20190201-I-1 131076385


In [36]:
# Print the first few lines from the first loan acquisition data file.

number_of_lines = 3
with open(sflp_squeue_files[0], 'r') as sflp_squeue_file:
    for x in range(number_of_lines):
        print(sflp_squeue_file.readline())

1549079401|21290343|PENDING|mkandes|use300|gpu-shared|1|7|5G|gpu:p100:1|2019-02-01T16:47:32|2019-02-02T05:29:44|0.0|3.0|10361|Resources|n/a||/home/mkandes/software/other/ahnold/build/comet/molecular-dynamics/gromacs/build-gromacs-tmpi-cuda.slurm

1549079401|21290385|PENDING|mkandes|use300|gpu-shared|1|7|5G|gpu:p100:1|2019-02-01T16:59:01|2019-02-02T05:30:46|0.0|3.0|10350|Priority|n/a||/home/mkandes/software/other/ahnold/build/comet/molecular-dynamics/gromacs/build-gromacs-tmpi-omp-cuda.slurm

1549079401|21291549|PENDING|mkandes|use300|gpu-shared|1|7|5G|gpu:p100:1|2019-02-01T17:38:01|2019-02-02T06:10:57|0.0|3.0|10311|Priority|n/a||/home/mkandes/software/other/ahnold/build/comet/molecular-dynamics/gromacs/build-gromacs-mpi-omp-cuda.slurm



In [37]:
# Define an ordered dictionary of column datatypes for the SFLP loan 
# acquisition dataset.

sflp_squeue_columns = collections.OrderedDict()
sflp_squeue_columns['UNIX_TIME'] = 'object'
sflp_squeue_columns['JOBID'] = 'category'
sflp_squeue_columns['STATE'] = 'category'
sflp_squeue_columns['USER'] = 'category'
sflp_squeue_columns['ACCOUNT'] = 'category'
sflp_squeue_columns['PARTITIAN'] = 'category'
sflp_squeue_columns['NODES'] = 'Int64'
sflp_squeue_columns['CPUSS'] ='Int64'
sflp_squeue_columns['MIN_MEMORY'] = 'category' #check if this is correct
sflp_squeue_columns['GRES'] = 'category'
sflp_squeue_columns['SUBMIT_TIME'] = 'object'
sflp_squeue_columns['START_TIME'] = 'object'
sflp_squeue_columns['TIME'] = 'float64'
sflp_squeue_columns['TIME_LIMIT'] = 'float64'
sflp_squeue_columns['PRIORITY'] = 'Int64'
sflp_squeue_columns['REASON'] = 'category'
sflp_squeue_columns['EXEC_HOST'] = 'category'
sflp_squeue_columns['NODELIST'] = 'category'
sflp_squeue_columns['COMMAND'] = 'category'

In [38]:
%%time
# Dask
# 
# Read in all of the loan acquisition data into a Pandas DataFrame 
# (sflp_acq_pdf) via the distributed dask.dataframe.read_csv method.

print(sflp_squeue_files)
print(*sflp_squeue_columns.keys())

dask_client = dask.distributed.Client(n_workers=26, threads_per_worker=1)

sflp_squeue_ddf = dask.dataframe.read_csv(sflp_squeue_files, 
                                       names=[*sflp_squeue_columns.keys()], 
                                       delimiter="|",
                                       dtype= sflp_squeue_columns)
sflp_squeue_ddf = dask_client.persist(sflp_squeue_ddf)
sflp_squeue_pdf = sflp_squeue_ddf.compute()
dask_client.close()

['/oasis/scratch/comet/caolinnh/temp_project/dataJup/SQUEUE/1549079401-20190201-I-1']
UNIX_TIME JOBID STATE USER ACCOUNT PARTITIAN NODES CPUSS MIN_MEMORY GRES SUBMIT_TIME START_TIME TIME TIME_LIMIT PRIORITY REASON EXEC_HOST NODELIST COMMAND


/opt/miniconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


CPU times: user 3.36 s, sys: 799 ms, total: 4.16 s
Wall time: 25 s


In [39]:
# Pandas
#
# Check datatypes of the loan acquisition Pandas DataFrame (sflp_acq_pdf).
#sflp_squeue_pdf["TIME_LIMIT"].fillna(0)
sflp_squeue_pdf.dtypes
#print(sflp_squeue_pdf['TIME_LIMIT'])

UNIX_TIME        object
JOBID          category
STATE          category
USER           category
ACCOUNT        category
PARTITIAN      category
NODES             Int64
CPUSS             Int64
MIN_MEMORY     category
GRES           category
SUBMIT_TIME      object
START_TIME       object
TIME            float64
TIME_LIMIT      float64
PRIORITY          Int64
REASON         category
EXEC_HOST      category
NODELIST       category
COMMAND        category
dtype: object

In [40]:
# Pandas
#
# Convert the date-related object datatypes in the loan acquisition
# Pandas DataFrame (sflp_acq_pdf) to datetime datatypes.
from datetime import datetime
#print(sflp_squeue_pdf['NODES'])
#print(sflp_squeue_pdf['JOBID'])
sflp_squeue_pdf['UNIX_TIME'] = pandas.to_datetime(
    sflp_squeue_pdf['UNIX_TIME'], unit="s")
sflp_squeue_pdf['SUBMIT_TIME'] = pandas.to_datetime(
    sflp_squeue_pdf['SUBMIT_TIME'], format= ("%Y-%m-%d"+"T"+"%H:%M:%S") )
sflp_squeue_pdf['START_TIME'] = pandas.to_datetime(
    sflp_squeue_pdf['START_TIME'], format= ("%Y-%m-%d"+"T"+"%H:%M:%S") )
#sflp_squeue_pdf['TIME_LIMIT'] = pandas.to_datetime(
    #sflp_squeue_pdf['TIME_LIMIT'], format= ("%d-%H:%M:%S") )
#if sflp_squeue_pdf['TIME'].item == "0:00":
    #sflp_squeue_pdf['TIME'].item = pandas.to_datetime(sflp_squeue_pdf['TIME'], format= ("%H:%M") )
#sflp_squeue_pdf
#else:
    #sflp_squeue_pdf['TIME'].item = pandas.to_datetime(sflp_squeue_pdf['TIME'], format= ("%d-%H:%M:%S") )
#df['hour'] = pd.to_datetime(df['time'], format='%H:%M').dt.hour
#sflp_squeue_pdf['first_payment_date'] = pandas.to_datetime(
    #sflp_squeue_pdf['first_payment_date'], format='%m/%Y')

In [41]:
# Pandas
#
# Re-check the datatypes of the loan acquisition Pandas DataFrame
# (sflp_acq_pdf) after completion of the object-to-datetime conversion. 
#sflp_squeue_pdf["TIME_LIMIT"] = pandas.to_numeric(sflp_squeue_pdf["TIME_LIMIT"], downcast="float")

sflp_squeue_pdf.dtypes

UNIX_TIME      datetime64[ns]
JOBID                category
STATE                category
USER                 category
ACCOUNT              category
PARTITIAN            category
NODES                   Int64
CPUSS                   Int64
MIN_MEMORY           category
GRES                 category
SUBMIT_TIME    datetime64[ns]
START_TIME     datetime64[ns]
TIME                  float64
TIME_LIMIT            float64
PRIORITY                Int64
REASON               category
EXEC_HOST            category
NODELIST             category
COMMAND              category
dtype: object

In [45]:
%%time 
# cuDF
#
# Create a copy of the loan acquisition Pandas DataFrame (sflp_acq_pdf)
# in the memory of a single NVIDIA GPU by creating a cuDF DataFrame
# (sflp_acq_cudf) from the Pandas DataFrame.

#sflp_squeue_cudf = cudf.DataFrame.from_pandas(sflp_squeue_pdf)
hehe = 0

for i in sflp_squeue_pdf['TIME_LIMIT']:
    if (type(i) != int) and (type(i) != float):
        if i == "0:00":
            i = 0.0 
        elif len(i) == 10:
            w = ((float(i[0])*24) + float(i[2:3]) + (float(i[5:6])/60))
            sflp_squeue_pdf['TIME_LIMIT']= sflp_squeue_pdf['TIME_LIMIT'].replace(i, w)
        elif len(i) == 8:
            w = (float(i[0:2]) + (float(i[4:5])/60))
            sflp_squeue_pdf['TIME_LIMIT']= sflp_squeue_pdf['TIME_LIMIT'].replace(i, w)
        elif len(i) == 7:
            w = (float(i[0]) + (float(i[2:3])/60))
            sflp_squeue_pdf['TIME_LIMIT']= sflp_squeue_pdf['TIME_LIMIT'].replace(i, w)
        elif len(i) == 5:
            w = (float(i[0:1])/60)
            sflp_squeue_pdf['TIME_LIMIT']= sflp_squeue_pdf['TIME_LIMIT'].replace(i, w)
        elif i == "":
            w = 0.0
            hehe = hehe + 1
            sflp_squeue_pdf['TIME_LIMIT']= sflp_squeue_pdf['TIME_LIMIT'].replace(i, w)
        elif len(i) == 4:
            w = (float(i[0])/60)
            sflp_squeue_pdf['TIME_LIMIT']= sflp_squeue_pdf['TIME_LIMIT'].replace(i, w)
        else:
            print(i + str(type(i)) )
            
print(str(hehe))
        

3.0<class 'str'>
3.0<class 'str'>
3.0<class 'str'>
3.0<class 'str'>
8.0<class 'str'>
8.0<class 'str'>
8.0<class 'str'>
8.0<class 'str'>
8.0<class 'str'>
8.0<class 'str'>
8.0<class 'str'>
8.0<class 'str'>
8.0<class 'str'>
8.0<class 'str'>
23.666666666666668<class 'str'>
23.666666666666668<class 'str'>
23.666666666666668<class 'str'>
23.666666666666668<class 'str'>
23.666666666666668<class 'str'>
23.666666666666668<class 'str'>
23.666666666666668<class 'str'>
23.666666666666668<class 'str'>
23.666666666666668<class 'str'>
4.5<class 'str'>
4.0<class 'str'>


ValueError: could not convert string to float: 'N'

In [ ]:
for x in sflp_squeue_pdf['TIME']:
    if (type(x) != int) and (type(x) != float):
        if x == "0:00":
            y = 0
            sflp_squeue_pdf['TIME']= sflp_squeue_pdf['TIME'].replace(x, y)
        elif len(x) == 10:
            y = ((int(x[0])*24) + int(x[2:3]) + (float(x[5:6])/60))
            for q in sflp_squeue_pdf['TIME']:
                if (type(q) != int) and (type(q) != float):
                    if q[0:6] == x[0:6]:
                        sflp_squeue_pdf['TIME']= sflp_squeue_pdf['TIME'].replace(q, y)
        elif len(x) == 8:
            y = (int(x[0:2]) + (float(x[4:5])/60))
            for q in sflp_squeue_pdf['TIME']:
                if (type(q) != int) and (type(q) != float):
                    if q[0:5] == x[0:5]:
                        sflp_squeue_pdf['TIME']= sflp_squeue_pdf['TIME'].replace(q, y)
        elif len(x) == 7:
            y = (int(x[0]) + (float(x[2:3])/60))
            for q in sflp_squeue_pdf['TIME']:
                if (type(q) != int) and (type(q) != float):
                    if q[0:3] == x[0:3]:
                        sflp_squeue_pdf['TIME']= sflp_squeue_pdf['TIME'].replace(q, y)
        elif len(x) == 5:
            y = (float(x[0:1])/60)
            for q in sflp_squeue_pdf['TIME']:
                if (type(q) != int) and (type(q) != float):
                    if q[0:1] == x[0:1]:
                        sflp_squeue_pdf['TIME']= sflp_squeue_pdf['TIME'].replace(q, y)
        elif len(x) == 4:
            y = (float(x[0])/60)
            for q in sflp_squeue_pdf['TIME']:
                if (type(q) != int) and (type(q) != float):
                    if q[0] == x[0]:
                        sflp_squeue_pdf['TIME']= sflp_squeue_pdf['TIME'].replace(q, y)

sflp_squeue_pdf.dtypes

In [26]:
# Pandas
#
# Inspect the first few rows of the loan acquisition Pandas DataFrame
# (sflp_acq_pdf).
print(sflp_squeue_pdf['TIME_LIMIT'])

sflp_squeue_pdf.dtypes

0              3.0
1              3.0
2              3.0
3              3.0
4        0.0166667
5        0.0666667
6        0.0333333
7        0.0333333
8        0.0666667
9        0.0666667
10       0.0666667
11       0.0333333
12       0.0333333
13       0.0333333
14       0.0666667
15       0.0666667
16       0.0333333
17       0.0333333
18       0.0666667
19       0.0666667
20       0.0666667
21       0.0666667
22       0.0666667
23       0.0666667
24       0.0666667
25       0.0666667
26       0.0666667
27       0.0666667
28       0.0666667
29       0.0666667
           ...    
13585    0.0666667
13586    0.0333333
13587    0.0166667
13588    0.0333333
13589    0.0333333
13590    0.0333333
13591    0.0333333
13592    0.0333333
13593    0.0333333
13594    0.0333333
13595    0.0333333
13596    0.0333333
13597    0.0333333
13598    0.0333333
13599    0.0666667
13600    0.0666667
13601    0.0666667
13602    0.0666667
13603    0.0666667
13604    0.0333333
13605    0.0333333
13606    0.0

UNIX_TIME      datetime64[ns]
JOBID                category
STATE                category
USER                 category
ACCOUNT              category
PARTITIAN            category
NODES                   Int64
CPUSS                   Int64
MIN_MEMORY           category
GRES                 category
SUBMIT_TIME    datetime64[ns]
START_TIME     datetime64[ns]
TIME                  float64
TIME_LIMIT             object
PRIORITY                Int64
REASON               category
EXEC_HOST            category
NODELIST             category
COMMAND              category
dtype: object

In [25]:
# cuDF
#
# Compare the first few rows of the loan acquisition cuDF DataFrame
# (sflp_acq_cudf) against those of the Pandas DataFrame (sflp_acq_pdf).

#sflp_squeue_cudf.head()

### SACCT Data

In [35]:
# List all of the files from the SACCT dataset to be analyzed
# as well as the size of each file in bytes.

for sflp_sacct_file in sflp_sacct_files:
    print(os.path.basename(sflp_sacct_file), os.stat(sflp_sacct_file).st_size)


18456165-I-1000.txt 754438


In [36]:
# Print the first few lines from the first loan acquisition data file.

number_of_lines = 3
with open(sflp_sacct_files[0], 'r') as sflp_sacct_file:
    for x in range(number_of_lines):
        print(sflp_sacct_file.readline())

18456165|18456165|519982|10059|16183|comet|sp_hse06_Li4|yixuan11|ddp309|ddp309|||compute|normal|1|0||0|0|384|Unknown|120Gn|||04:00:00|0|CANCELLED by 507569|0:0|0:0|2018-08-09T16:07:23|2018-08-09T16:07:23|2019-12-03T11:07:58|2019-12-03T11:07:58|04:00:00|00:00:00|480-20:00:35|00:00:00|||||INVALID|3068007552|00:00:00|||00:00:00|0||||||||||||||||||||||||||None assigned

20122149|20122149|521942|10670|18997|comet|rep4_step7_17|hoang029|csm107|csm107|||compute|normal|1|0||0|0|48|Unknown|120Gn|||1-00:00:00|1301|CANCELLED by 507569|0:0|0:0|2018-11-13T15:41:25|2018-11-13T15:41:25|2019-07-09T12:59:11|2019-07-09T12:59:11|1-00:00:00|00:00:00|237-20:17:46|00:00:00|||||11416-14:12:48|986393568|00:00:00|||00:00:00|0||||||||||||||||||||||||||None assigned

20122150|20122150|521942|10670|18997|comet|rep4_step7_18|hoang029|csm107|csm107|||compute|normal|1|0||0|0|48|Unknown|120Gn|||1-00:00:00|1301|CANCELLED by 507569|0:0|0:0|2018-11-13T15:41:25|2018-11-13T15:41:25|2019-07-09T12:59:11|2019-07-09T12:59:11|

In [43]:
# Define an ordered dictionary of column datatypes for the SFLP loan 
# acquisition dataset.

#yo whats up

sflp_sacct_columns = collections.OrderedDict()
sflp_sacct_columns['JobIDRaw'] = 'Int64'
sflp_sacct_columns['JobID'] = 'Int64'
sflp_sacct_columns['UID'] = 'Int64'
sflp_sacct_columns['GID'] = 'Int64'
sflp_sacct_columns['AssocID'] = 'Int64'
sflp_sacct_columns['Cluster'] = 'category'
sflp_sacct_columns['JobName'] = 'category'
sflp_sacct_columns['User'] ='category'
sflp_sacct_columns['Group'] = 'category' #check if this is correct
sflp_sacct_columns['Account'] = 'category'
sflp_sacct_columns['Reservation'] = 'object'
sflp_sacct_columns['ReservationID'] = 'object'
sflp_sacct_columns['Partition'] = 'category'
sflp_sacct_columns['QOS'] = 'object'
sflp_sacct_columns['QOSRAW'] = 'Int64'
sflp_sacct_columns['NNODES'] = 'Int64'
sflp_sacct_columns['NTasks'] = 'Int64'
sflp_sacct_columns['NCPUS'] = 'Int64'
sflp_sacct_columns['AllocCPUS'] = 'Int64'
sflp_sacct_columns['ReqCPUS'] = 'Int64'
sflp_sacct_columns['ReqCPUSFreq'] = 'object'
sflp_sacct_columns['ReqMem'] = 'object'
sflp_sacct_columns['ReqGRES'] = 'object'
sflp_sacct_columns['AllocGRES'] = 'object'
sflp_sacct_columns['TimeLimit'] = 'object'
sflp_sacct_columns['Priority'] = 'Int64'
sflp_sacct_columns['State'] = 'category'
sflp_sacct_columns['ExitCode'] = 'object'
sflp_sacct_columns['DerivedExitCode'] = 'object'
sflp_sacct_columns['Sumbit'] = 'object'
sflp_sacct_columns['Eligible'] = 'object'
sflp_sacct_columns['Start'] = 'object'
sflp_sacct_columns['End'] = 'object'
sflp_sacct_columns['Time'] = 'object'
sflp_sacct_columns['Elapsed'] = 'object'
sflp_sacct_columns['Reserved'] = 'object'
sflp_sacct_columns['Suspended'] = 'object'
sflp_sacct_columns['AveCPU'] = 'object'
sflp_sacct_columns['MinCPU'] = 'object'
sflp_sacct_columns['MinCPUNode'] = 'object'
sflp_sacct_columns['MinCPUTask'] = 'object'
sflp_sacct_columns['ResvCPU'] = 'object'
sflp_sacct_columns['ResvCPURaw'] = 'object'
sflp_sacct_columns['TotalCPU'] = 'object'
sflp_sacct_columns['SystemCPU'] = 'object'
sflp_sacct_columns['UserCPU'] = 'object'
sflp_sacct_columns['CPUTime'] = 'object'
sflp_sacct_columns['CPUTimeRaw'] = 'object'
sflp_sacct_columns['AveCPUFreq'] = 'object'
sflp_sacct_columns['AveDiskRead'] = 'object'
sflp_sacct_columns['MaxDiskRead'] = 'object'
sflp_sacct_columns['MaxDiskReadNode'] = 'object'
sflp_sacct_columns['MaxDiskReadTask'] = 'object'
sflp_sacct_columns['AveDiskWrite'] = 'object'
sflp_sacct_columns['MaxDiskWrite'] = 'object'
sflp_sacct_columns['MaxDiskWriteNode'] = 'object'
sflp_sacct_columns['MaxDiskWriteTask'] = 'object'
sflp_sacct_columns['AvePages'] = 'object'
sflp_sacct_columns['MaxPages'] = 'object'
sflp_sacct_columns['MaxPagesNode'] = 'object'
sflp_sacct_columns['MaxPagesTask'] = 'object'
sflp_sacct_columns['AveRSS'] = 'object'
sflp_sacct_columns['MaxRSS'] = 'object'
sflp_sacct_columns['MaxRSSNode'] = 'object'
sflp_sacct_columns['MaxRSSTask'] = 'object'
sflp_sacct_columns['AveVMSize'] = 'object'
sflp_sacct_columns['MaxVMSize'] = 'object'
sflp_sacct_columns['MaxVMSizeNode'] = 'object'
sflp_sacct_columns['MaxVMSizeTask'] = 'object'
sflp_sacct_columns['ConsumedEnergy'] = 'object'
sflp_sacct_columns['ConsumedEnergyRaw'] = 'object'
sflp_sacct_columns['Layout'] = 'object'
sflp_sacct_columns['Comment'] = 'category'
sflp_sacct_columns['NodeList'] = 'object'

In [44]:
%%time
# Dask
# 
# Read in all of the loan acquisition data into a Pandas DataFrame 
# (sflp_acq_pdf) via the distributed dask.dataframe.read_csv method.

print(sflp_sacct_files)
print(*sflp_sacct_columns.keys())

dask_client = dask.distributed.Client(n_workers=26, threads_per_worker=1)

sflp_squeue_ddf = dask.dataframe.read_csv(sflp_sacct_files, 
                                       names=[*sflp_sacct_columns.keys()], 
                                       delimiter="|",
                                       dtype= "object" )#sflp_sacct_columns)
sflp_sacct_ddf = dask_client.persist(sflp_squeue_ddf)
sflp_sacct_pdf = sflp_sacct_ddf.compute()
dask_client.close()

['/home/caolinnh/REHS_code/dataJup/SACCT/18456165-I-1000.txt']
JobIDRaw JobID UID GID AssocID Cluster JobName User Group Account Reservation ReservationID Partition QOS QOSRAW NNODES NTasks NCPUS AllocCPUS ReqCPUS ReqCPUSFreq ReqMem ReqGRES AllocGRES TimeLimit Priority State ExitCode DerivedExitCode Sumbit Eligible Start End Time Elapsed Reserved Suspended AveCPU MinCPU MinCPUNode MinCPUTask ResvCPU ResvCPURaw TotalCPU SystemCPU UserCPU CPUTime CPUTimeRaw AveCPUFreq AveDiskRead MaxDiskRead MaxDiskReadNode MaxDiskReadTask AveDiskWrite MaxDiskWrite MaxDiskWriteNode MaxDiskWriteTask AvePages MaxPages MaxPagesNode MaxPagesTask AveRSS MaxRSS MaxRSSNode MaxRSSTask AveVMSize MaxVMSize MaxVMSizeNode MaxVMSizeTask ConsumedEnergy ConsumedEnergyRaw Layout Comment NodeList
CPU times: user 1min 26s, sys: 23.2 s, total: 1min 50s
Wall time: 3min 50s


In [45]:
# Pandas
#
# Re-check the datatypes of the loan acquisition Pandas DataFrame
# (sflp_acq_pdf) after completion of the object-to-datetime conversion. 

sflp_sacct_pdf.dtypes

JobIDRaw             object
JobID                object
UID                  object
GID                  object
AssocID              object
Cluster              object
JobName              object
User                 object
Group                object
Account              object
Reservation          object
ReservationID        object
Partition            object
QOS                  object
QOSRAW               object
NNODES               object
NTasks               object
NCPUS                object
AllocCPUS            object
ReqCPUS              object
ReqCPUSFreq          object
ReqMem               object
ReqGRES              object
AllocGRES            object
TimeLimit            object
Priority             object
State                object
ExitCode             object
DerivedExitCode      object
Sumbit               object
                      ...  
SystemCPU            object
UserCPU              object
CPUTime              object
CPUTimeRaw           object
AveCPUFreq          

## Data Exploration and Manipulation

### Filtering and Querying Data

In [66]:
%%time
# Pandas
#
# How many loans were made for more than $400,000?
import re

#pandas.Series.str.count(sflp_squeue_pdf.query('TIME_LIMIT'),'N')
#sflp_squeue_pdf["count"]= sflp_squeue_pdf["TIME_LIMIT"].str.count("N", re.I)
#series = sflp_squeue_pdf["TIME_LIMIT"]
#count = series.str.count("N")
sflp_squeue_pdf[sflp_squeue_pdf['TIME_LIMIT'].str.contains("N", na = False)]
#sflp_squeue_pdf
#print(count)
#print(sflp_squeue_pdf["TIME_LIMIT"])

CPU times: user 225 ms, sys: 14.2 ms, total: 239 ms
Wall time: 223 ms


,UNIX_TIME,JOBID,STATE,USER,ACCOUNT,PARTITIAN,NODES,CPUSS,MIN_MEMORY,GRES,SUBMIT_TIME,START_TIME,TIME,TIME_LIMIT,PRIORITY,REASON,EXEC_HOST,NODELIST,COMMAND
565,2019-02-02 03:50:01,21218905,RUNNING,qwzhan,dku134,compute,1,24,120G,(null),2019-01-27 20:23:55,2019-01-27 20:23:56,98.033333,None,9642,None,comet-11-61,comet-11-61,/home/qwzhan/Chevron/Frac-HEX8/run.sb
566,2019-02-02 03:50:01,21218911,RUNNING,qwzhan,dku134,compute,1,24,120G,(null),2019-01-27 20:26:23,2019-01-27 20:26:38,98.038889,None,9640,None,comet-11-67,comet-11-67,./run.sb
3403,2019-02-01 18:15:01,21218905,RUNNING,qwzhan,dku134,compute,1,24,120G,(null),2019-01-27 20:23:55,2019-01-27 20:23:56,97.083333,None,9075,None,comet-11-61,comet-11-61,/home/qwzhan/Chevron/Frac-HEX8/run.sb
3404,2019-02-01 18:15:01,21218911,RUNNING,qwzhan,dku134,compute,1,24,120G,(null),2019-01-27 20:26:23,2019-01-27 20:26:38,97.072222,None,9073,None,comet-11-67,comet-11-67,./run.sb
5046,2019-02-01 16:10:01,21218905,RUNNING,qwzhan,dku134,compute,1,24,120G,(null),2019-01-27 20:23:55,2019-01-27 20:23:56,97.066667,None,8947,None,comet-11-61,comet-11-61,/home/qwzhan/Chevron/Frac-HEX8/run.sb
5048,2019-02-01 16:10:01,21218911,RUNNING,qwzhan,dku134,compute,1,24,120G,(null),2019-01-27 20:26:23,2019-01-27 20:26:38,97.072222,None,8945,None,comet-11-67,comet-11-67,./run.sb
6760,2019-02-02 00:30:01,21218905,RUNNING,qwzhan,dku134,compute,1,24,120G,(null),2019-01-27 20:23:55,2019-01-27 20:23:56,98.000000,None,9434,None,comet-11-61,comet-11-61,/home/qwzhan/Chevron/Frac-HEX8/run.sb
6761,2019-02-02 00:30:01,21218911,RUNNING,qwzhan,dku134,compute,1,24,120G,(null),2019-01-27 20:26:23,2019-01-27 20:26:38,98.005556,None,9432,None,comet-11-67,comet-11-67,./run.sb
8610,2019-02-01 17:10:01,21218905,RUNNING,qwzhan,dku134,compute,1,24,120G,(null),2019-01-27 20:23:55,2019-01-27 20:23:56,97.066667,None,9010,None,comet-11-61,comet-11-61,/home/qwzhan/Chevron/Frac-HEX8/run.sb
8611,2019-02-01 17:10:01,21218911,RUNNING,qwzhan,dku134,compute,1,24,120G,(null),2019-01-27 20:26:23,2019-01-27 20:26:38,97.072222,None,9008,None,comet-11-67,comet-11-67,./run.sb


In [67]:
%%time
# cuDF
#
# How many loans were made for more than $400,000?
sflp_squeue_pdf
#len(sflp_acq_cudf.query('original_unpaid_principal_balance > 400000'))

CPU times: user 10 µs, sys: 2 µs, total: 12 µs
Wall time: 24.6 µs


,UNIX_TIME,JOBID,STATE,USER,ACCOUNT,PARTITIAN,NODES,CPUSS,MIN_MEMORY,GRES,SUBMIT_TIME,START_TIME,TIME,TIME_LIMIT,PRIORITY,REASON,EXEC_HOST,NODELIST,COMMAND
0,2019-02-02 03:50:01,21290343,PENDING,mkandes,use300,gpu-shared,1,7,5G,gpu:p100:1,2019-02-01 16:47:32,2019-02-02 05:29:44,0.000000,3.0,10361,Resources,NaN,NaN,/home/mkandes/software/other/ahnold/build/come...
1,2019-02-02 03:50:01,21290385,PENDING,mkandes,use300,gpu-shared,1,7,5G,gpu:p100:1,2019-02-01 16:59:01,2019-02-02 05:30:46,0.000000,3.0,10350,Priority,NaN,NaN,/home/mkandes/software/other/ahnold/build/come...
2,2019-02-02 03:50:01,21291549,PENDING,mkandes,use300,gpu-shared,1,7,5G,gpu:p100:1,2019-02-01 17:38:01,2019-02-02 06:10:57,0.000000,3.0,10311,Priority,NaN,NaN,/home/mkandes/software/other/ahnold/build/come...
3,2019-02-02 03:50:01,21291548,PENDING,mkandes,use300,gpu-shared,1,7,5G,gpu:p100:1,2019-02-01 17:37:21,2019-02-02 06:02:31,0.000000,3.0,10311,Priority,NaN,NaN,/home/mkandes/software/other/ahnold/build/come...
4,2019-02-02 03:50:01,21281902,PENDING,cipres,sds121,gpu,1,28,120G,gpu:p100:4,2019-01-31 21:56:12,2019-02-02 05:30:31,0.000000,120.0,9339,Resources,NaN,NaN,./_batch_command.run
5,2019-02-02 03:50:01,21281153,PENDING,jaehbae,mia307,gpu,2,8,64G,gpu:p100:4,2019-01-31 19:08:23,2019-02-02 09:57:11,0.000000,48.0,9228,Resources,NaN,NaN,/home/jaehbae/fargo3d-twhya/gpurun.sh
6,2019-02-02 03:50:01,21286776,PENDING,cipres,sds121,gpu,1,28,120G,gpu:p100:4,2019-02-01 06:53:21,2019-02-02 10:59:54,0.000000,20.0,8802,Resources,NaN,NaN,./_batch_command.run
7,2019-02-02 03:50:01,21286773,PENDING,cipres,sds121,gpu,1,28,120G,gpu:p100:4,2019-02-01 06:53:01,2019-02-02 09:58:23,0.000000,20.0,8802,Resources,NaN,NaN,./_batch_command.run
8,2019-02-02 03:50:01,21287086,PENDING,hemma011,umn113,gpu,2,48,120G,gpu:k80:4,2019-02-01 07:45:04,2019-02-02 08:14:24,0.000000,48.0,8778,Resources,NaN,NaN,/oasis/scratch/comet/hemma011/temp_project/Col...
9,2019-02-02 03:50:01,21287089,PENDING,hemma011,umn113,gpu,2,48,120G,gpu:k80:4,2019-02-01 07:45:54,2019-02-02 14:19:09,0.000000,48.0,8777,Resources,NaN,NaN,/oasis/scratch/comet/hemma011/temp_project/Col...


In [ ]:
%%time
# Pandas
#
# How many loans were made in California for more than $400,000?

len(sflp_acq_pdf.query('property_state == "CA" and \
                        original_unpaid_principal_balance > 400000'))

In [ ]:
%%time
# cuDF (FAIL)
#
# How many loans were made in California for more than $400,000?

len(sflp_acq_cudf.query('property_state == "CA" and \
                        original_unpaid_principal_balance > 400000'))

In [ ]:
# cuDF
#
# Check the list of categories in the property_state column. Find CA.

sflp_acq_cudf.property_state.cat.categories

In [ ]:
%%time 
# cuDF
#
# It appears that cuDF still does not support literal string comparisons
# in the cudf.DataFrame.query() method. e.g., instead, you must use the
# numerical index one of the categories in a category datatype. 
#
# How many loans were made in California (4) for more than $400,000?

len(sflp_acq_cudf.query('property_state == 4 and \
                         original_unpaid_principal_balance > 400000'))

In [ ]:
%%time 
# Pandas
# 
# How many loans were made in either California, New York, or Florida
# for more than $400,000 to principal borrowers who had a credit score
# of less than 650?

len(sflp_acq_pdf.query('(property_state == "CA" or \
                         property_state == "NY" or \
                         property_state =="FL") and \
                         original_unpaid_principal_balance > 400000 and \
                         borrower_credit_score_at_origination < 650'))

In [ ]:
%%time
# cuDF
# 
# How many loans were made in either California (4), New York (34), or 
# Florida (9) for more than $400,000 to principal borrowers who had a 
# credit score of less than 650?

len(sflp_acq_cudf.query('(property_state == 4 or \
                          property_state == 34 or \
                          property_state == 9) and \
                          original_unpaid_principal_balance > 400000 and \
                          borrower_credit_score_at_origination < 650'))

### Grouping Data

In [ ]:
%%time
# Pandas   .count(Int64)
#
# What was the total number of loans originated per month? 

total_number_of_loans_originated_per_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['loan_identifier'].count().to_frame()

In [ ]:
%%time
# cuDF    .count(int64)
#
# What was the total number of loans originated per month?
# .count(Int64)

total_number_of_loans_originated_per_month_cudf = sflp_acq_cudf.groupby(
    'origination_date')['loan_identifier'].count().to_frame()

In [ ]:
%%time
# Pandas   .sum(Int64)
# 
# What was the total unpaid principal balance originated per month (in
# billions of dollars)?

total_unpaid_principal_balance_originated_per_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['original_unpaid_principal_balance'].sum().to_frame().div(1000000000)

In [ ]:
%%time
# cuDF    .sum(int64)   (FAIL)
# 
# What was the total unpaid principal balance originated per month (in
# billions of dollars)?

total_unpaid_principal_balances_originated_per_month_cudf  = sflp_acq_cudf.groupby(
    'origination_date')['original_unpaid_principal_balance'].sum().to_frame().div(1000000000)

In [ ]:
%%time
# Pandas   .sum(Int64)
# 
# What was the total unpaid principal balance originated per month?

total_unpaid_principal_balance_originated_per_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['original_unpaid_principal_balance'].sum().to_frame()

In [ ]:
%%time
# cuDF    .sum(int64)
# 
# What was the total unpaid principal balance originated per month?

total_unpaid_principal_balance_originated_per_month_cudf = sflp_acq_cudf.groupby(
    'origination_date')['original_unpaid_principal_balance'].sum().to_frame()

In [ ]:
%%time
# Pandas   .mean(Int64)
# 
# What was the average unpaid principal balance at originiation by month?

average_unpaid_principal_balance_at_origination_by_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['original_unpaid_principal_balance'].mean().to_frame()

In [ ]:
%%time
# cuDF    .mean(int64)    
# 
# What was the average unpaid principal balance at originiation by month?

average_unpaid_principal_balance_at_origination_by_month_cudf = sflp_acq_cudf.groupby(
    'origination_date')['original_unpaid_principal_balance'].mean().to_frame()

In [ ]:
%%time
# Pandas   .mean(float64)
#
# What was the average loan to value ratio at originiation by month?

average_loan_to_value_at_origination_by_month_pdf  = sflp_acq_pdf.groupby(
    'origination_date')['original_loan_to_value'].mean().to_frame()

In [ ]:
%%time
# cuDF    .mean(float64)
#
# What was the average loan to value ratio at originiation by month?

average_loan_to_value_at_origination_by_month_cudf  = sflp_acq_cudf.groupby(
    'origination_date')['original_loan_to_value'].mean().to_frame()

In [ ]:
%time
# Pandas
#
# What was the average credit score of a borrower at originiation?

average_credit_score_at_origination_by_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['borrower_credit_score_at_origination'].mean().to_frame()

In [ ]:
%time
# cuDF
#
# What was the average credit score of a borrower at originiation?

average_credit_score_at_origination_by_month_cudf = sflp_acq_cudf.groupby(
    'origination_date')['borrower_credit_score_at_origination'].mean().to_frame()

In [ ]:
%%time
# Pandas
#
# What was the average debt-to-income ratio of a borrower at origination?

average_debt_to_income_at_origination_by_month_pdf  = sflp_acq_pdf.groupby(
    'origination_date')['original_debt_to_income_ratio'].mean().to_frame()

In [ ]:
%%time
# cuDF
#
# What was the average debt-to-income ratio of a borrower at origination?

average_debt_to_income_at_origination_by_month_cudf  = sflp_acq_cudf.groupby(
    'origination_date')['original_debt_to_income_ratio'].mean().to_frame()

In [ ]:
%%time
# Pandas
# 
# What was the average interest rate on a loan at origination?

average_interest_rate_at_origination_by_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['original_interest_rate'].mean().to_frame()

In [ ]:
%%time
# cuDF
# 
# What was the average interest rate on a loan at origination?

average_interest_rate_at_origination_by_month_cudf = sflp_acq_cudf.groupby(
    'origination_date')['original_interest_rate'].mean().to_frame()

### Concatenating Data

In [ ]:
%%time
# Pandas
#
# Combine the monthly summary statistics series created above into a
# single pandas DataFrame.

sflp_acq_monthly_summary_pdf = pandas.concat([total_number_of_loans_originated_per_month_pdf, 
                                      total_unpaid_principal_balance_originated_per_month_pdf, 
                                      average_unpaid_principal_balance_at_origination_by_month_pdf, 
                                      average_loan_to_value_at_origination_by_month_pdf, 
                                      average_credit_score_at_origination_by_month_pdf, 
                                      average_debt_to_income_at_origination_by_month_pdf, 
                                      average_interest_rate_at_origination_by_month_pdf], axis=1)

In [ ]:
%%time
# cuDF
#
# Combine the monthly summary statistics series created above into a
# single pandas DataFrame.

sflp_acq_monthly_summary_cudf = cudf.concat([total_number_of_loans_originated_per_month_cudf, 
                                      total_unpaid_principal_balance_originated_per_month_cudf, 
                                      average_unpaid_principal_balance_at_origination_by_month_cudf, 
                                      average_loan_to_value_at_origination_by_month_cudf, 
                                      average_credit_score_at_origination_by_month_cudf, 
                                      average_debt_to_income_at_origination_by_month_cudf, 
                                      average_interest_rate_at_origination_by_month_cudf], axis=1)

## Cluster Analysis and Classification